In [1]:
import numpy as np
import pandas as pd
%pylab inline
from flask import Flask, json, jsonify, render_template

Populating the interactive namespace from numpy and matplotlib


In [2]:
class TrieNode(object):
  
    def __init__(self, char: str):
        self.char = char
        self.children = []
        self.word_finished = False
        
def add(root, word: str):
    word = word.lower().replace('ё', 'е')  #lower- преобразует все символы в нижний регистр, replace меняет ё на е
    node = root
    for char in word:
        found_in_child = False
        for child in node.children:
            if child.char == char:
                node = child
                found_in_child = True
                break
        if not found_in_child:
            new_node = TrieNode(char)
            node.children.append(new_node)
            node = new_node
    node.word_finished = True
    
def prefix_descent(root, prefix: str):
    node = root
    for char in prefix:
        char_not_found = True
        for child in node.children:
            if child.char == char:
                char_not_found = False
                node = child
                break
        if char_not_found:
            return False
    return node


def trie_print(node, string, level, list_of_strings):
    if node.word_finished:
        res_string = ''.join(string)
        list_of_strings.append(res_string)
    else:
        for child in node.children:
            string.insert(level, child.char)
            trie_print(child, string.copy(), level+1, list_of_strings)
            string.pop()


def suggester(root, prefix):
    prefix = prefix.lower()
    node = prefix_descent(root, prefix)
    list_of_strings = []
    string = []
    if type(node) == bool:
        list_of_strings.append(prefix)
    else:
        trie_print(node, string, 0, list_of_strings)
        for i in range(len(list_of_strings)):
            list_of_strings[i] = prefix + list_of_strings[i]
    return list_of_strings   

In [3]:
def rating(root, word , number: int):
    string = suggester(root, word)
    sugg_data = pd.DataFrame(columns =  ["name", "rating"])
    n = len(data)
    i = 0
    while i < n:
        for s in string:
            if data.at[i, 'name'] == s:
                sugg_data = pd.DataFrame(np.array([[data.loc[i]['name'], data.loc[i]['rating']]]), columns=['name', 'rating']).append(sugg_data, ignore_index=True)
        i += 1
    sugg_data['rating'] = sugg_data['rating'].apply(lambda x: int(x))
    sugg_data = sugg_data.sort_values(['rating'], ascending=[0])
    sugg_data = sugg_data[:number]
    #string = sugg_data['name']
    return sugg_data['name']

In [4]:
data = pd.read_json('/home/darya/cook_dataset.json')
data['name'] = data['name'].apply(lambda x: x.lower().replace('ё', 'е'))
data.head()

,cuisine,ingredients,name,rating
0,francuzskaya,"[Яблоко, Финики, Мед, Лимонный сок, Ваниль, Со...",коблер,343
1,kitayskaya,"[Филе куриной грудки, Апельсины, Соевый соус, ...",куриные кусочки в апельсиновом соусе с чили,13
2,kitayskaya,"[Белое сухое вино, Соевый соус, Чеснок, Жидкий...",курица в кисло-сладком маринаде,66
3,armyanskaya,"[Вода, Пшеничная мука, Соль, Сливочное масло, ...",армянский лаваш,569
4,kitayskaya,"[Стебель сельдерея, Зеленый лук перья, Соевый ...",салат из сельдерея по‑китайски,62


In [5]:
root = TrieNode('*')

In [6]:
for i in data['name']:
    add(root, i)

In [ ]:


app = Flask(__name__)
app.config['JSON_AS_ASCII'] = False


@app.route('/get_suggest/<prefix>', methods=['GET'])
def suggestions(prefix):
    suggest = rating(root, prefix, 10)
    sug_string = ''
    for i in suggest:
        sug_string += i + '<br/>'
    return json.dumps(sug_string[:-1])

@app.route('/')
def hello():
    return 'Начни вводить название блюда'



if __name__ == "__main__":
    app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Nov/2018 22:59:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2018 22:59:46] "GET /get_suggest/%D0%BA%D0%B0%D1%80 HTTP/1.1" 200 -
